## Working With the Applications API

### Prerequisites:

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

#### Running the following cell will prompt you for the `Client ID` and `Client Secret` and create a `client` instance that can be used to communicate with the Fairnow APIs.

In [ ]:
import json
from getpass import getpass
import httpx
from httpx_auth import OAuth2ClientCredentials

client_id = "{client_id}" # Replace with your Client Id
client_secret = getpass("Client Secret")
fairnow_token_endpoint = "https://auth.fairnow.ai/oauth2/token"

auth = OAuth2ClientCredentials(
    token_url=fairnow_token_endpoint,
    client_id=client_id,
    client_secret=client_secret,
)

fairnow_base_url = "https://api.fairnow.ai/v2"
client = httpx.Client(base_url=fairnow_base_url, auth=auth)

#### Create A New AI Application

#### At a minimum an AI Application requires an Application Name, Source and Type. Valid Sources are `internal` or `vendor`.  The response will give you a `application_id` and `application_version`, save these for use in other notebooks.

In [ ]:
application_route = "/application"
request_body = {
    "application_name": "API Client Test Application",  # Can provide your own name
    "application_type": "Test Type",                    # Can provide your own type description
    "application_source": "internal",                   # Must be 'internal' or 'vendor'
    "application_version": None,                        # Optional, will default to `1.0`
    "vendor_id": None                                   # Optional if 'application_source' is 'internal', required if source is 'vendor'
}

application_id = None
application_version = None

response = client.post(application_route, json=request_body)

if response.status_code == 200:
    application = response.json()
    
    application_id = application.get("application_id")
    application_version = application.get("application_version")
    application_name = application.get("application_name")
    print("Created new Application:")
    print(json.dumps(response.json(), indent=4))
else:
    print(f"Error: {response.status_code} - {response.text}")


#### With the `application_id` and `application_version` you can now lookup the new Application:

In [ ]:
applications_route = "/applications/"
query_parameters = {
    "application_id": application_id,
    "application_version": application_version,
}

response = client.get(applications_route, params=query_parameters)

if response.status_code == 200:
    print("API Response:")
    print(json.dumps(response.json(), indent=4))
else:
    print(f"Error: {response.status_code} - {response.text}")


#### Save the `application_id` and `application_version` so that you can perform other compliance tasks for the application like approving it for deployment, testing for biases or adding evidence that a control has been met.

In [ ]:
print(f"Application ID: {application_id}")
print(f"Application Version: {application_version}")